In [3]:
import svv_connector as svv   # SEE BELOW FOR ITS DEFINITION

1. Can you think of how you would modify the class you created in HW 6 to use context management?

In [1]:
class SVVConnection(object):

    
    def __enter__(self):
        self.Connection = svv.get_svv_connection()
        self.my_cursor = self.Connection.cursor()
        return self
    
    def __exit__(self, exc_type, exc_value, tb):
        self.my_cursor.close()
        self.Connection.close()
    
            
    def executeQuery(self, query):
        ''' Executes a query on SVV using an opened cursor my_cursor
        '''
        
        if self.my_cursor is None:
            print ("Cursor not open. Call getCursor first.")
            return
        
        try:
            self.my_cursor.execute(query)
        except Exception as e: 
            print(e)
            
    def getBatchQueryResults(self,  batch_size=1):
        ''' Returns a generator from which query results can be sources in batch_size
        '''
        res = 0
        
        try:
            while res != []:
                res = self.my_cursor.fetchmany(batch_size)
                yield res
                
        except Exception as e: 
            print(e)

In [4]:
# Outside of the WITH, SVVConnection DOES exist as a class, but the connection to the SVV is CLOSED and so is the cursor

with SVVConnection() as my_connection:    #<-The conect management is done here with WITH
    query = "Select Top 10 * from sandbox.pwc_summary_hc_2018_es;"
    
    my_connection.executeQuery(query)
    my_results = my_connection.getBatchQueryResults(10)
    print (next(my_results))
    print (next(my_results))
    print (next(my_results)) #<-exausted here

[('0b69df82-b190-4450-9f51-83f75cf1b876', 'M', 63, 'Mr', 'aa.ad.ma.me', 'PO10 7', 'EMSWORTH', 87, 'Active', 'PO10 7NH', '72', '3', '4', '00', '11', '12', '3', '0', '4', '12', '0', '0', '13', '14', '4', '02', '04', '1', '0', '11', '3', '10', '4', '0', '0', '3', 21500, '2', 21500, 'ITV.com', None, Decimal('6113.7560000000'), 'Long Lost Family: What Happened Next', 'FACTUAL', '{"id":"REAL LIVES","ranking":"1"}', 0, 0, 0, 0, 0, 'evening', None, Decimal('0.0000'), 'VERY LOW', 'Suburbanites', 'Semi-detached suburbia', 'White suburban communities'), ('15a2ae04-bb7f-47de-86d7-5bac25cd521a', 'F', 65, 'Ms', 'aa.ad.ha.hc.hw.wa.wo', 'GL2 0', 'GLOUCESTER', 6, 'Active', 'GL2 0SQ', '53', '3', '3', '00', '11', '06', '3', '1', '4', '06', '0', '0', '10', '09', '4', '08', '01', '0', '1', '11', '4', '09', '2', '1', '0', '5', 40600, '4', 40600, 'Android', Decimal('16315.2900000000'), Decimal('31599.9320000000'), '100 Years Younger in 21 Days', 'DRAMA', '{"id":"PERIOD AND HISTORICAL","ranking":"1"}', 0, 0, 

StopIteration: 

In [5]:
# This shows that the cursor is closed

query = "Select Top 10 * from sandbox.pwc_summary_hc_2018_es;"
my_connection.executeQuery(query)

cursor already closed


### _/_/_/_/_/_/_/_/_/_/_/_/_/_/_/_/_/_/_/_/_/_/_/

import psycopg2


def get_svv_connection():

    con=psycopg2.connect(dbname= 'svv', host='svv-rs-prod-bi.cjddijbnvfpr.eu-west-1.redshift.amazonaws.com', \
                         port= 5439, user= 'my_user_name', password= 'my_password')
    
    return con